### Olympic data transformation using spark
* Author : Dixit Prajapati
* Date : August 2024

### Introduction
This notebook demonstrates the process of transforming Olympic data using Apache Spark. The dataset includes various metrics related to Olympic athletes and their performances across different events. The transformations aim to prepare the data for further analysis.


In [ ]:
# Configuring and mounting an Azure Data Lake Storage (ADLS) container to a Databricks filesystem (DBFS) using OAuth authentication

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "", # Update client ID 
"fs.azure.account.oauth2.client.secret": "", # Update secret key
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_Id_Update/oauth2/token"} # Update tenant_Id


dbutils.fs.mount(
source = "abfss://container@storageaccountname.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/olympicc",
extra_configs = configs)
  

In [ ]:
# Importing function and datatypes from pyspark.

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
# Using Databricks' magic command %fs to interact with the Databricks File System (DBFS)

%fs
ls "/mnt/olympicc"

In [ ]:
spark

In [ ]:
%fs
ls "dbfs:/mnt/olympicc/raw-data/"

In [ ]:
# Load the medals data from a CSV file into a Spark DataFrame

athletes = spark.read.csv("/mnt/olympicc/raw-data/athletes.csv", header=True, inferSchema=True)
coaches = spark.read.csv("/mnt/olympicc/raw-data/coaches.csv", header=True, inferSchema=True)
entriesgender = spark.read.csv("/mnt/olympicc/raw-data/entriesgender.csv", header=True, inferSchema=True)
medals = spark.read.csv("/mnt/olympicc/raw-data/medals.csv", header=True, inferSchema=True)
teams = spark.read.csv("/mnt/olympicc/raw-data/teams.csv", header=True, inferSchema=True)

In [ ]:
# Show the first few rows of the athletes DataFrame
athletes.show()

In [ ]:
# Print the schema of the athletes DataFrame to understand its structure
athletes.printSchema()

In [ ]:
coaches.show()

In [ ]:
coaches.printSchema()

In [ ]:
entriesgender.show()

In [ ]:
entriesgender.printSchema()

In [ ]:
medals.show()

In [ ]:
medals.printSchema()

In [ ]:
teams.show()

In [ ]:
teams.printSchema()

In [ ]:
# Find the top countries with the highest number of gold medals
medals.show()

In [ ]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team/NOC","Gold").show()

In [ ]:
# Calculate the average number of entries by gender for each discipline
from pyspark.sql.functions import round
average_entries_by_gender = entriesgender.withColumn('Avg_Female', entriesgender['Female'] / entriesgender['Total']).withColumn('Avg_Male',entriesgender['Male'] / entriesgender['Total']).withColumn('Avg_Female', round('Avg_Female',2)).withColumn('Avg_Male', round('Avg_Male',2))
average_entries_by_gender.show() 

In [ ]:
# Created 2 partition for each file
# athletes.repartition(2).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/athletes')
# coaches.repartition(2).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/coaches')
# entriesgender.repartition(2).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/entriesgender')
# medals.repartition(2).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/medals')
# teams.repartition(2).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/teams')

In [ ]:
athletes.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/athletes')
coaches.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/coaches')
entriesgender.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/entriesgender')
medals.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/medals')
teams.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/olympicc/transformed-data/teams')

In [ ]:
%fs
ls "/mnt/olympicc/transformed-data/"

In [ ]:
%fs
ls "/mnt/olympicc/transformed-data/athletes/"

In [ ]:
# Load the athletes data from a CSV file into a Spark DataFrame

athletes = spark.read.csv("/mnt/olympicc/transformed-data/athletes/part-00000-tid-8364754138149393228-d1a2f989-0312-4682-9bff-8d851edb0bb1-279-1-c000.csv", header=True, inferSchema=True)
athletes.show()


### Conclusion
This notebook successfully demonstrated how to process and analyze Olympic data using Apache Spark. The transformed data provides valuable insights that can be used for further exploration or decision-making.
